In [2]:
import pandas as pd
from sqlalchemy import create_engine

# Database connection string
engine = create_engine('postgresql://uvg_user:uvg_password@db:5432/health_data')

# Simple query to test the foundation layer
try:
    df = pd.read_sql("SELECT 1 as connection_status", engine)
    print("Connection Successful! Your Biomedical Data Stack is ready.")
    print(df)
except Exception as e:
    print(f"Connection Failed: {e}")

Connection Successful! Your Biomedical Data Stack is ready.
   connection_status
0                  1


In [4]:
df = pd.read_sql("""
SELECT patient_id, sex, birth_date, created_at
FROM patient
ORDER BY patient_id;
""", engine)

df


,patient_id,sex,birth_date,created_at
0,1,F,1990-05-12,2026-01-15 19:15:06.417200
1,2,M,1984-11-03,2026-01-15 19:15:06.417200
2,3,O,2001-07-21,2026-01-15 19:15:06.417200


In [5]:
pd.read_sql("SELECT sex, COUNT(*) n FROM patient GROUP BY sex ORDER BY n DESC;", engine)


,sex,n
0,O,1
1,M,1
2,F,1


In [3]:
pd.read_sql("""
SELECT
  p.subject_id,
  p.external_id,
  p.full_name,
  a.hadm_id,
  a.admission_type,
  a.admittime,
  a.dischtime,
  a.hospital_expire_flag
FROM patients p
JOIN admissions a ON a.subject_id = p.subject_id
ORDER BY a.admittime;
""", engine)

,subject_id,external_id,full_name,hadm_id,admission_type,admittime,dischtime,hospital_expire_flag


In [1]:
import psycopg2
import pandas as pd

conn = psycopg2.connect(
    host="db",
    port=5432,
    database="health_data",
    user="uvg_user",
    password="uvg_password"
)


In [4]:
import psycopg2
import pandas as pd

conn = psycopg2.connect(
    host="db",
    port=5432,
    database="health_data",
    user="uvg_user",
    password="uvg_password"
)

query = """
SELECT
    p.external_id,
    p.full_name,
    COUNT(*) AS n_admissions
FROM admissions a
JOIN patients p
    ON a.subject_id = p.subject_id
GROUP BY
    p.subject_id,
    p.external_id,
    p.full_name
HAVING COUNT(*) > 1;

"""

df = pd.read_sql(query, conn)
df


/tmp/ipykernel_227/198260886.py:28: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


,external_id,full_name,n_admissions
0,MRN-0006,Luc??a Herrera,2
1,MRN-0001,Ana L??pez,2


In [5]:
import psycopg2
import pandas as pd

# Conexión (ya confirmada que funciona)
conn = psycopg2.connect(
    host="db",
    port=5432,
    database="health_data",
    user="uvg_user",
    password="uvg_password"
)

query = """
SELECT
    a.hadm_id,
    p.external_id,
    p.full_name,
    EXTRACT(EPOCH FROM (a.dischtime - a.admittime)) / 86400.0
        AS length_of_stay_days
FROM admissions a
JOIN patients p
    ON a.subject_id = p.subject_id
ORDER BY length_of_stay_days DESC;
"""

df = pd.read_sql(query, conn)
df



/tmp/ipykernel_227/4265309551.py:26: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


,hadm_id,external_id,full_name,length_of_stay_days
0,5,MRN-0004,Jos?? Mart??nez,8.375000
1,3,MRN-0002,Carlos P??rez,7.500000
2,9,MRN-0007,Miguel Castillo,6.333333
3,1,MRN-0001,Ana L??pez,5.250000
4,2,MRN-0001,Ana L??pez,3.958333
5,6,MRN-0005,Alex Rivera,3.541667
6,7,MRN-0006,Luc??a Herrera,3.020833
7,4,MRN-0003,Mar??a G??mez,2.916667
8,8,MRN-0006,Luc??a Herrera,2.833333
9,10,MRN-0008,Sof??a Morales,2.541667


In [6]:
import psycopg2
import pandas as pd

# Conexión a la base
conn = psycopg2.connect(
    host="db",
    port=5432,
    database="health_data",
    user="uvg_user",
    password="uvg_password"
)

query = """
SELECT
    a.hadm_id,
    p.external_id,
    p.full_name,
    MAX(le.value_num) AS max_creatinine
FROM labevents le
JOIN d_labitems d
    ON le.labitem_id = d.labitem_id
JOIN admissions a
    ON le.hadm_id = a.hadm_id
JOIN patients p
    ON a.subject_id = p.subject_id
WHERE d.label = 'Creatinine'
GROUP BY a.hadm_id, p.external_id, p.full_name
ORDER BY max_creatinine DESC;
"""

df = pd.read_sql(query, conn)
df


/tmp/ipykernel_227/2451285899.py:31: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


,hadm_id,external_id,full_name,max_creatinine
0,3,MRN-0002,Carlos P??rez,3.1
1,9,MRN-0007,Miguel Castillo,1.8
2,1,MRN-0001,Ana L??pez,1.6
3,8,MRN-0006,Luc??a Herrera,1.4
4,7,MRN-0006,Luc??a Herrera,0.9


In [7]:
import psycopg2
import pandas as pd

conn = psycopg2.connect(
    host="db",
    port=5432,
    database="health_data",
    user="uvg_user",
    password="uvg_password"
)

query = """
SELECT
    a.hadm_id,
    p.external_id,
    p.full_name,
    MAX(CASE 
        WHEN d.label = 'White Blood Cells' THEN le.value_num 
    END) AS max_wbc,
    MAX(CASE 
        WHEN d.label = 'Lactate' THEN le.value_num 
    END) AS max_lactate
FROM labevents le
JOIN d_labitems d
    ON le.labitem_id = d.labitem_id
JOIN admissions a
    ON le.hadm_id = a.hadm_id
JOIN patients p
    ON a.subject_id = p.subject_id
WHERE d.label IN ('White Blood Cells', 'Lactate')
GROUP BY a.hadm_id, p.external_id, p.full_name
HAVING
    MAX(CASE WHEN d.label = 'White Blood Cells' THEN le.value_num END) > 12
    OR
    MAX(CASE WHEN d.label = 'Lactate' THEN le.value_num END) > 2
ORDER BY a.hadm_id;
"""

df = pd.read_sql(query, conn)
df


/tmp/ipykernel_227/1888346748.py:39: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


,hadm_id,external_id,full_name,max_wbc,max_lactate
0,3,MRN-0002,Carlos P??rez,18.4,3.8
1,5,MRN-0004,Jos?? Mart??nez,14.2,2.2
2,6,MRN-0005,Alex Rivera,12.5,NaN
3,10,MRN-0008,Sof??a Morales,13.0,NaN
